In [39]:
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

train_topics = [u'freedom_of_speech', 
                u'human_rights', u'law',  
                u'world_international', 
                u'economy', u'culture', 
               ]

dev_topics = [  
    u'health_and_medicine', u'society',  u'science_and_technology',u'gender', u'education', 
]

test_topics = [
    u'politics',
    u'digital_freedom', 
    u'sports_and_entertainments', 
    u'religion', 
    u'environment', 
    u'philosophy', 
    u'ethics'
]

split_id = {}

total_train_size = 0; 
total_dev_size = 0; 
total_test_size = 0; 
for c in claims:
    cId = c['cId']
    topics = c['topics']
    claim_text = c['text']
#     for topic_text in topics:
    train_int = len(intersection(topics, train_topics))
    dev_int = len(intersection(topics, dev_topics))
#     assert train_int == 0 or dev_int == 0 
    if train_int > 0: 
        total_train_size += 1
        split_id[cId] = "train"
    elif dev_int > 0: 
        total_dev_size += 1
        split_id[cId] = "dev"
    else: 
        total_test_size += 1
        split_id[cId] = "test"
#         if topic_text in train_topics:
#             total_train_size += 1
#             break  

print(total_train_size)
print(total_dev_size)
print(total_test_size)
print(len(claims))
print(total_train_size * 1.0 / len(claims))
print(total_dev_size * 1.0 / len(claims))
print(total_test_size * 1.0 / len(claims))
print((total_train_size + total_test_size + total_dev_size)* 1.0 / len(claims))


import json
with open('/Users/daniel/ideaProjects/perspective/data/lucene_cach/dataset_split_v0.2.json', 'w') as outfile:
    json.dump(split_id, outfile)

541
139
227
907
0.596471885336
0.153252480706
0.250275633958
1.0


In [ ]:
# creating clusters of claims that do not share any perspectives

In [21]:
# OLD: create stance training data

# import json 

# with open('../data/dataset/perspectrum_with_answers_v0.1.json') as data:
#     all_annotations = json.load(data)

# with open('../data/dataset/perspective_pool_v0.1.json') as data:
#     perspectives = json.load(data)

# p_map = {}
# for p in perspectives: 
#     p_map[p['pId']] = p['text']


# # create pairs of claims and perspectices and save them in a csv file 

# import csv

# import sys
# reload(sys)
# sys.setdefaultencoding('utf8')


# def save_in_file(claims, file_name): 
#     with open('../data/dataset/' + file_name, mode='w') as employee_file:
#         write = csv.writer(employee_file, delimiter='\t', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#         for c in claims: 
#             claim_text = c["text"]
#             for p in c["perspectives"]:                
#                 for pid in p["pids"]: 
#                     if "SUPPORT" in p['stance_label_3']: 
#                         p_text = p_map[pid]
#                         print(pid)
#                         print(p_text)
#                         write.writerow([str(1), str(0), str(0), claim_text, p_text])  
#                     if "UNDERMINE" in p['stance_label_3']: 
#                         p_text = p_map[pid]
#                         write.writerow([str(0), str(0), str(0), claim_text, p_text])  
# split_idx = int(0.7 * len(all_annotations))  
        
# save_in_file(all_annotations[0:split_idx], 'perspective_stances/train.tsv')
# save_in_file(all_annotations[1 + split_idx:], 'perspective_stances/dev.tsv')

In [19]:
# Create stance training data

import json 

with open('../data/dataset/perspectrum_with_answers_v0.2.json', encoding='utf-8') as data:
    all_annotations = json.load(data)

with open('../data/dataset/perspective_pool_v0.2.json', encoding='utf-8') as data:
    perspectives = json.load(data)

with open('../data/dataset/dataset_split_v0.2.json', encoding='utf-8') as data:
    split = json.load(data)
    

p_map = {}
for p in perspectives: 
    p_map[p['pId']] = p['text']
    
# create pairs of claims and perspectices and save them in a csv file 

import csv

# import sys
# reload(sys)
# sys.setdefaultencoding('utf8')

def save_in_file(claims, file_name): 
    with open('../data/dataset/' + file_name, mode='w') as employee_file:
        write = csv.writer(employee_file, delimiter='\t', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for c in claims: 
            claim_text = c["text"]
            for p in c["perspectives"]:                
                for pid in p["pids"]: 
                    if "SUPPORT" in p['stance_label_3']: 
                        p_text = p_map[pid]
#                         print(pid)
#                         print(p_text)
                        write.writerow([str(1), str(0), str(0), claim_text, p_text])  
                    if "UNDERMINE" in p['stance_label_3']: 
                        p_text = p_map[pid]
                        write.writerow([str(0), str(0), str(0), claim_text, p_text])  

train_claims = [_c for _c in all_annotations if split[str(_c['cId'])] == 'train']
test_claims = [_c for _c in all_annotations if split[str(_c['cId'])] == 'test']
dev_claims = [_c for _c in all_annotations if split[str(_c['cId'])] == 'dev']

save_in_file(train_claims, 'perspective_stances/train.tsv')
save_in_file(test_claims, 'perspective_stances/test.tsv')
save_in_file(dev_claims, 'perspective_stances/dev.tsv')

In [34]:
# Create relevance training data

import json 
import random
import query_elasticsearch as es

with open('../data/dataset/perspectrum_with_answers_v0.2.json', encoding='utf-8') as data:
    all_annotations = json.load(data)

with open('../data/dataset/perspective_pool_v0.2.json', encoding='utf-8') as data:
    perspectives = json.load(data)

with open('../data/dataset/dataset_split_v0.2.json', encoding='utf-8') as data:
    split = json.load(data)
    

all_pid_list = []
    
c_map ={}
for c in all_annotations:
    c_map[c["cId"]] = c["text"]
    
p_map = {}
for p in perspectives: 
    pid = p['pId']
    all_pid_list.append(pid)
    p_map[pid] = p['text']

related_p_map = {}
for c in all_annotations:
    cands = []
    cid = c["cId"]
    for cluster in c["perspectives"]:
        cands += cluster["pids"]
    
    related_p_map[cid] = set(cands)
    


# create pairs of claims and perspectices and save them in a csv file 

import csv

# import sys
# reload(sys)
# sys.setdefaultencoding('utf8')

def save_in_file(claims, file_name, test_or_dev=False): 
    with open('../data/dataset/' + file_name, mode='w') as employee_file:
        write = csv.writer(employee_file, delimiter='\t', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for c in claims: 
            count = 0
            cid = c["cId"]
            claim_text = c["text"]
            lucene_results = es.get_perspective_from_pool(claim_text, 50)
            
            if test_or_dev:
                pid_set = [_pid for _text, _pid, _score in lucene_results]
            else:
                pid_set = related_p_map[cid]
                
            for pid in pid_set:
                if pid in related_p_map[cid]:
                    p_text = p_map[pid]
                    write.writerow([str(1), str(cid), str(pid), claim_text, p_text])
                    count += 1
            
            for _text, pid, _score in lucene_results:
                if pid not in related_p_map[cid]:
                    p_text = p_map[pid]
                    write.writerow([str(0), str(cid), str(pid), claim_text , p_text])
                    count -= 1
                    if count <= 0:
                        break
                    

train_claims = [_c for _c in all_annotations if split[str(_c['cId'])] == 'train']
test_claims = [_c for _c in all_annotations if split[str(_c['cId'])] == 'test']
dev_claims = [_c for _c in all_annotations if split[str(_c['cId'])] == 'dev']

save_in_file(train_claims, 'perspective_relevance/train.tsv')
save_in_file(test_claims, 'perspective_relevance/test.tsv', test_or_dev=True)
save_in_file(dev_claims, 'perspective_relevance/dev.tsv', test_or_dev=True)

In [36]:
# WIP - WIP - WIP - Evaluate BERT results on perpsective relevance
import csv

raw_score = '../data/dataset/perspective_relevance/test_raw_score.csv'
test_data = '../data/dataset/perspective_relevance/test.tsv'

test_cids = []
for line in open(test_data):
    line = line.rstrip()
    if line:
        test_cids.append(line.split('\t')[1])
        
test_cids.pop(0) # somehow the bert script ignores the first instance during reading


reader = csv.DictReader(open(raw_score, 'r'))
count = 0
for line in reader:
    cid = test_cids[count]
    und_score = float(line['undermine_score']) 
    sup_score = float(line['support_score']) 
    label = 1 if sup_score > und_score else 0
    
    count += 1
    break;


-0.7695837
